# Exercise 4: Backtest

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tabulate
import jupyter as jp

In [3]:
df = pd.read_csv("data/AAPL.csv", header='infer', delimiter=',', index_col='Date', parse_dates=True)

In [ ]:
df.info()

In [ ]:
# 1. drop the missing values and compute the daily future return on the adjusted close price
df.dropna(inplace=True)
returns = (df['Adj Close'].shift(-1) - df['Adj Close']) / df['Adj Close']
returns

##### 2. Random bool series

* The information is this series should be interpreted this way:

    * On the 2010-01-01 I receive 1 before the market closes meaning that, if I trust the signal, the close price of day d+1 will increase. I should buy the stock before the market closes.
    * On the 2010-01-02 I receive 0 before the market closes meaning that,, if I trust the signal, the close price of day d+1 will not increase. I should not buy the stock.


In [ ]:
random_booleans = np.random.randint(0,2,len(df.index))
bool_serie = pd.Series(random_booleans, index=df.index,name="Signal")
bool_serie.head()

##### 3. Signal backtesting

In [ ]:
st = df.merge(bool_serie,left_index=True,right_index=True)
# Calculate daily return: (Close_d+1 - Close_d) / Close_d
st['Next_Close'] = st['Close'].shift(-1)
st['Daily_Return'] = (st['Next_Close'] - st['Close']) / st['Close']
st['PnL'] = st['Signal'] * st['Daily_Return'].fillna(0)

strategy_return = st['PnL']
print(strategy_return)

##### 4. strategy processing

In [8]:
total_earned = st['PnL'].sum()
total_invested = st['Signal'].sum()

# Calculate the return of the strategy
strategy_return = (total_earned - total_invested) / total_invested if total_invested != 0 else 0

In [ ]:
st['Signal_Q5'] = 1  # Always buy
st['PnL_Q5'] = st['Signal_Q5'] * st['Daily_Return'].fillna(0)

total_pnl_q3 = st['PnL'].sum()
total_pnl_q5 = st['PnL_Q5'].sum()

print(f"Total PnL for Q3 strategy: {total_pnl_q3:.4f}")
print(f"Total PnL for Q5 strategy: {total_pnl_q5:.4f}")

plt.figure(figsize=(10, 6))
plt.plot(st.index, st['PnL_Q5'], label='Q5: Always Buy', marker='o', linestyle='--')
plt.plot(st.index, st['PnL'], label='Q3: Signal Based', marker='x', linestyle='-')
plt.title('Daily PnL for Q5 (Always Buy) vs Q3 (Signal Based)')
plt.xlabel('Date')
plt.ylabel('Daily PnL')
plt.legend()
plt.grid(True)
plt.show()